In [1]:
import re
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [2]:
df = pd.read_csv('IPL 1st Innings.csv')

In [3]:
df.head()

,match_id,year,date,city,venue,team1,team2,toss_winner,toss_decision,innings,batting_team,bowling_team,batsman,non_striker,bowler,overs,balls,ball_no,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,runs,wickets,runs_last_5_overs,wickets_last_5_overs,winner,result,result_margin,player_of_match,playoff,knockout,final,method,players,umpire1,umpire2,final_score,total_wickets,target_overs,target_runs
0,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,1,0,1,1,0,0,NaN,NaN,NaN,legbyes,1,0,1,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
1,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,2,2,0,0,0,0,0,NaN,NaN,NaN,NaN,1,0,1,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
2,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,3,3,0,1,1,0,0,NaN,NaN,NaN,wides,2,0,2,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
3,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,4,3,0,0,0,0,0,NaN,NaN,NaN,NaN,2,0,2,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
4,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,5,4,0,0,0,0,0,NaN,NaN,NaN,NaN,2,0,2,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN


In [4]:
df.shape

(107664, 46)

In [5]:
df = df[df['overs'] > 4]

In [6]:
df.head()

,match_id,year,date,city,venue,team1,team2,toss_winner,toss_decision,innings,batting_team,bowling_team,batsman,non_striker,bowler,overs,balls,ball_no,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,runs,wickets,runs_last_5_overs,wickets_last_5_overs,winner,result,result_margin,player_of_match,playoff,knockout,final,method,players,umpire1,umpire2,final_score,total_wickets,target_overs,target_runs
32,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,5,1,31,1,0,1,0,0,NaN,NaN,NaN,NaN,61,0,59,0,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
33,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,Z Khan,5,2,32,0,0,0,0,1,71384.0,SC Ganguly,JH Kallis,NaN,61,1,59,1,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
34,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,RT Ponting,BB McCullum,Z Khan,5,3,33,0,0,0,0,0,NaN,NaN,NaN,NaN,61,1,59,1,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
35,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,RT Ponting,BB McCullum,Z Khan,5,4,34,0,0,0,0,0,NaN,NaN,NaN,NaN,61,1,59,1,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN
36,335982,2008,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,RT Ponting,BB McCullum,Z Khan,5,5,35,0,0,0,0,0,NaN,NaN,NaN,NaN,61,1,58,1,Kolkata Knight Riders,runs,140.0,BB McCullum,0,0,0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,222,3,NaN,NaN


In [7]:
df.shape

(80344, 46)

In [8]:
df = df[['year', 'venue', 'toss_decision', 'batting_team', 'bowling_team', 'overs', 'ball_no', 'runs', 'wickets', 'runs_last_5_overs', 
         'wickets_last_5_overs', 'playoff', 'knockout', 'final', 'final_score']]

In [9]:
dummy = ['venue', 'toss_decision', 'batting_team', 'bowling_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = False)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [10]:
columns = []
for i in df.columns:
    i = re.sub('\s', '_', i)
    i = re.sub('\.', '', i)
    i = re.sub('\,', '', i)
    i = re.sub('\-', '_', i)
    columns.append(i)
df.columns = columns

In [11]:
df = df[['year', 'batting_team_Chennai_Super_Kings', 'batting_team_Deccan_Chargers', 'batting_team_Delhi_Capitals', 'batting_team_Gujarat_Lions', 
         'batting_team_Kochi_Tuskers_Kerala', 'batting_team_Kolkata_Knight_Riders', 'batting_team_Mumbai_Indians', 'batting_team_Pune_Warriors', 
         'batting_team_Punjab_Kings', 'batting_team_Rajasthan_Royals', 'batting_team_Rising_Pune_Supergiant', 
         'batting_team_Royal_Challengers_Bangalore', 'batting_team_Sunrisers_Hyderabad', 'bowling_team_Chennai_Super_Kings', 
         'bowling_team_Deccan_Chargers', 'bowling_team_Delhi_Capitals', 'bowling_team_Gujarat_Lions', 'bowling_team_Kochi_Tuskers_Kerala', 
         'bowling_team_Kolkata_Knight_Riders', 'bowling_team_Mumbai_Indians', 'bowling_team_Pune_Warriors', 'bowling_team_Punjab_Kings', 
         'bowling_team_Rajasthan_Royals', 'bowling_team_Rising_Pune_Supergiant', 'bowling_team_Royal_Challengers_Bangalore', 
         'bowling_team_Sunrisers_Hyderabad', 'toss_decision_bat', 'toss_decision_field', 'venue_Arun_Jaitley_Stadium', 
         'venue_Barabati_Stadium', 'venue_Brabourne_Stadium', 'venue_Buffalo_Park', 'venue_De_Beers_Diamond_Oval', 
         'venue_Dr_DY_Patil_Sports_Academy', 'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium', 
         'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens', 'venue_Green_Park', 
         'venue_Himachal_Pradesh_Cricket_Association_Stadium', 'venue_Holkar_Cricket_Stadium', 
         'venue_JSCA_International_Stadium_Complex', 'venue_Kingsmead', 'venue_M_Chinnaswamy_Stadium', 'venue_MA_Chidambaram_Stadium', 
         'venue_Maharashtra_Cricket_Association_Stadium', 'venue_Narendra_Modi_Stadium', 'venue_Nehru_Stadium', 'venue_New_Wanderers_Stadium', 
         'venue_Newlands', 'venue_OUTsurance_Oval', 'venue_Punjab_Cricket_Association_IS_Bindra_Stadium', 
         'venue_Rajiv_Gandhi_International_Stadium', 'venue_Saurashtra_Cricket_Association_Stadium', 'venue_Sawai_Mansingh_Stadium', 
         'venue_Shaheed_Veer_Narayan_Singh_International_Stadium', 'venue_Sharjah_Cricket_Stadium', 'venue_Sheikh_Zayed_Stadium', 
         'venue_St_George\'s_Park', 'venue_SuperSport_Park', 'venue_Vidarbha_Cricket_Association_Stadium', 'venue_Wankhede_Stadium', 'overs', 
         'ball_no', 'runs', 'wickets', 'runs_last_5_overs', 'wickets_last_5_overs', 'playoff', 'knockout', 'final', 'final_score']]

In [12]:
df.shape

(80344, 72)

In [13]:
train = df[df['year'] <= 2019]
test = df[df['year'] >= 2020]

In [14]:
train.year.unique(), test.year.unique()

(array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
        2019], dtype=int64),
 array([2020, 2021], dtype=int64))

In [15]:
X_train = pd.DataFrame(train.drop(['year', 'final_score'], axis = 1), columns = train.drop(['year', 'final_score'], axis = 1).columns)
X_test = pd.DataFrame(test.drop(['year', 'final_score'], axis = 1), columns = test.drop(['year', 'final_score'], axis = 1).columns)
y_train = train['final_score']
y_test = test['final_score']

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((69134, 70), (11210, 70), (69134,), (11210,))

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [18]:
la = Lasso(alpha = 0.23)
la.fit(X_train, y_train)

Lasso(alpha=0.23)

In [19]:
ri = Ridge(alpha = 0.898)
ri.fit(X_train, y_train)

Ridge(alpha=0.898)

In [20]:
abr = AdaBoostRegressor(n_estimators = 50, learning_rate = 0.475)
abr.fit(X_train, y_train)

AdaBoostRegressor(learning_rate=0.475)

In [21]:
knr = KNeighborsRegressor(weights = 'uniform', n_neighbors = 10)
knr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [22]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
y_train_pred_la = la.predict(X_train)
y_test_pred_la = la.predict(X_test)
y_train_pred_ri = ri.predict(X_train)
y_test_pred_ri = ri.predict(X_test)
y_train_pred_abr = abr.predict(X_train)
y_test_pred_abr = abr.predict(X_test)
y_train_pred_knr = knr.predict(X_train)
y_test_pred_knr = knr.predict(X_test)

In [23]:
predictions = [y_test_pred_lr, y_test_pred_la, y_test_pred_ri, y_test_pred_abr, y_test_pred_knr]
columns = ['y_test_pred_lr', 'y_test_pred_la', 'y_test_pred_ri', 'y_test_pred_abr', 'y_test_pred_knr']
for col, pred in zip(columns, predictions):
    test[col] = pred
predictions = [y_train_pred_lr, y_train_pred_la, y_train_pred_ri, y_train_pred_abr, y_train_pred_knr]
columns = ['y_train_pred_lr', 'y_train_pred_la', 'y_train_pred_ri', 'y_train_pred_abr', 'y_train_pred_knr']
for col, pred in zip(columns, predictions):
    train[col] = pred

In [24]:
#lr and la

test['lr_la_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_la']) / 2
train['lr_la_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_la']) / 2
y_ensemble_train = train['lr_la_prediction'].values
y_ensemble_test = test['lr_la_prediction'].values
print('LR and LA Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

LR and LA Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6436349118620025
Test r2_score:  0.6505139523797315
----------------------------------------------------------------------------------------------------
Train RMSE:  17.59126038406875
Test RMSE:  17.90743397162226
----------------------------------------------------------------------------------------------------
Train MAE:  13.007725034018215
Test MAE:  13.982935725119964


In [25]:
#lr and ri

test['lr_ri_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_ri']) / 2
train['lr_ri_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_ri']) / 2
y_ensemble_train = train['lr_ri_prediction'].values
y_ensemble_test = test['lr_ri_prediction'].values
print('LR and RI Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

LR and RI Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6482904272911945
Test r2_score:  0.6218253370346456
----------------------------------------------------------------------------------------------------
Train RMSE:  17.47597746245229
Test RMSE:  18.627932331174595
----------------------------------------------------------------------------------------------------
Train MAE:  12.994918064256034
Test MAE:  14.555160156088595


In [26]:
#lr and abr

test['lr_abr_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_abr']) / 2
train['lr_abr_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_abr']) / 2
y_ensemble_train = train['lr_abr_prediction'].values
y_ensemble_test = test['lr_abr_prediction'].values
print('LR and ABR Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

LR and ABR Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.62081635005878
Test r2_score:  0.6043944753000006
----------------------------------------------------------------------------------------------------
Train RMSE:  18.145719066982394
Test RMSE:  19.05239651296611
----------------------------------------------------------------------------------------------------
Train MAE:  13.685557117803034
Test MAE:  14.988897896823454


In [27]:
#lr and knr

test['lr_knr_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_knr']) / 2
train['lr_knr_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_knr']) / 2
y_ensemble_train = train['lr_knr_prediction'].values
y_ensemble_test = test['lr_knr_prediction'].values
print('LR and KNR Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

LR and KNR Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7499793224418474
Test r2_score:  0.6578445976067815
----------------------------------------------------------------------------------------------------
Train RMSE:  14.734565896441184
Test RMSE:  17.718629953003255
----------------------------------------------------------------------------------------------------
Train MAE:  10.7222107448297
Test MAE:  13.498341255562313


In [28]:
#la and ri

test['la_ri_prediction'] = (test['y_test_pred_la'] + test['y_test_pred_ri']) / 2
train['la_ri_prediction'] = (train['y_train_pred_la'] + train['y_train_pred_ri']) / 2
y_ensemble_train = train['la_ri_prediction'].values
y_ensemble_test = test['la_ri_prediction'].values
print('la and ri Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

la and ri Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.643624523998787
Test r2_score:  0.6505053670894356
----------------------------------------------------------------------------------------------------
Train RMSE:  17.591516770399927
Test RMSE:  17.90765392257021
----------------------------------------------------------------------------------------------------
Train MAE:  13.007831548437409
Test MAE:  13.98307049473902


In [29]:
#la and abr

test['la_abr_prediction'] = (test['y_test_pred_la'] + test['y_test_pred_abr']) / 2
train['la_abr_prediction'] = (train['y_train_pred_la'] + train['y_train_pred_abr']) / 2
y_ensemble_train = train['la_abr_prediction'].values
y_ensemble_test = test['la_abr_prediction'].values
print('la and abr Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

la and abr Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6058301018951772
Test r2_score:  0.613230170872314
----------------------------------------------------------------------------------------------------
Train RMSE:  18.50082559077056
Test RMSE:  18.838431124114692
----------------------------------------------------------------------------------------------------
Train MAE:  13.940601598104301
Test MAE:  14.960342558859315


In [30]:
#la and knr

test['la_knr_prediction'] = (test['y_test_pred_la'] + test['y_test_pred_knr']) / 2
train['la_knr_prediction'] = (train['y_train_pred_la'] + train['y_train_pred_knr']) / 2
y_ensemble_train = train['la_knr_prediction'].values
y_ensemble_test = test['la_knr_prediction'].values
print('la and knr Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

la and knr Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.739004709499659
Test r2_score:  0.664164004871351
----------------------------------------------------------------------------------------------------
Train RMSE:  15.054478538831994
Test RMSE:  17.554241083329817
----------------------------------------------------------------------------------------------------
Train MAE:  10.915374031747083
Test MAE:  13.452317376846276


In [31]:
#ri and abr

test['ri_abr_prediction'] = (test['y_test_pred_ri'] + test['y_test_pred_abr']) / 2
train['ri_abr_prediction'] = (train['y_train_pred_ri'] + train['y_train_pred_abr']) / 2
y_ensemble_train = train['ri_abr_prediction'].values
y_ensemble_test = test['ri_abr_prediction'].values
print('ri and abr Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

ri and abr Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6208155442842923
Test r2_score:  0.6043950950001082
----------------------------------------------------------------------------------------------------
Train RMSE:  18.145738347019545
Test RMSE:  19.0523815905547
----------------------------------------------------------------------------------------------------
Train MAE:  13.685593413127707
Test MAE:  14.989143852292829


In [32]:
#ri and knr

test['ri_knr_prediction'] = (test['y_test_pred_ri'] + test['y_test_pred_knr']) / 2
train['ri_knr_prediction'] = (train['y_train_pred_ri'] + train['y_train_pred_knr']) / 2
y_ensemble_train = train['ri_knr_prediction'].values
y_ensemble_test = test['ri_knr_prediction'].values
print('ri and knr Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

ri and knr Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7499733842419766
Test r2_score:  0.6578332795589774
----------------------------------------------------------------------------------------------------
Train RMSE:  14.734740874524261
Test RMSE:  17.718923004992284
----------------------------------------------------------------------------------------------------
Train MAE:  10.722258633126318
Test MAE:  13.498651223485949


In [33]:
#abr and knr

test['abr_knr_prediction'] = (test['y_test_pred_abr'] + test['y_test_pred_knr']) / 2
train['abr_knr_prediction'] = (train['y_train_pred_abr'] + train['y_train_pred_knr']) / 2
y_ensemble_train = train['abr_knr_prediction'].values
y_ensemble_test = test['abr_knr_prediction'].values
print('abr and knr Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

abr and knr Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7052237190011126
Test r2_score:  0.6108382721129666
----------------------------------------------------------------------------------------------------
Train RMSE:  15.999103545710943
Test RMSE:  18.896592549509997
----------------------------------------------------------------------------------------------------
Train MAE:  12.102935093129883
Test MAE:  14.853858983606237


In [34]:
#lr, la, and ri

test['lr_la_ri_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_la'] + test['y_test_pred_ri']) / 3
train['lr_la_ri_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_la'] + train['y_train_pred_ri']) / 3
y_ensemble_train = train['lr_la_ri_prediction'].values
y_ensemble_test = test['lr_la_ri_prediction'].values
print('lr, la and ri Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

lr, la and ri Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6462167632528133
Test r2_score:  0.6432529319469749
----------------------------------------------------------------------------------------------------
Train RMSE:  17.527420541294152
Test RMSE:  18.092502596516876
----------------------------------------------------------------------------------------------------
Train MAE:  12.980017383558451
Test MAE:  14.122818815577022


In [ ]:
#lr, la, and ri

test['lr_la_ri_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_la'] + test['y_test_pred_ri']) / 3
train['lr_la_ri_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_la'] + train['y_train_pred_ri']) / 3
y_ensemble_train = train['lr_la_ri_prediction'].values
y_ensemble_test = test['lr_la_ri_prediction'].values
print('lr, la and ri Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

In [23]:
predictions = [y_test_pred_lr, y_test_pred_la, y_test_pred_ri, y_test_pred_abr, y_test_pred_knr]
columns = ['y_test_pred_lr', 'y_test_pred_la', 'y_test_pred_ri', 'y_test_pred_abr', 'y_test_pred_knr']
for col, pred in zip(columns, predictions):
    test[col] = pred

In [24]:
test.head()

,year,batting_team_Chennai_Super_Kings,batting_team_Deccan_Chargers,batting_team_Delhi_Capitals,batting_team_Gujarat_Lions,batting_team_Kochi_Tuskers_Kerala,batting_team_Kolkata_Knight_Riders,batting_team_Mumbai_Indians,batting_team_Pune_Warriors,batting_team_Punjab_Kings,batting_team_Rajasthan_Royals,batting_team_Rising_Pune_Supergiant,batting_team_Royal_Challengers_Bangalore,batting_team_Sunrisers_Hyderabad,bowling_team_Chennai_Super_Kings,bowling_team_Deccan_Chargers,bowling_team_Delhi_Capitals,bowling_team_Gujarat_Lions,bowling_team_Kochi_Tuskers_Kerala,bowling_team_Kolkata_Knight_Riders,bowling_team_Mumbai_Indians,bowling_team_Pune_Warriors,bowling_team_Punjab_Kings,bowling_team_Rajasthan_Royals,bowling_team_Rising_Pune_Supergiant,bowling_team_Royal_Challengers_Bangalore,bowling_team_Sunrisers_Hyderabad,toss_decision_bat,toss_decision_field,venue_Arun_Jaitley_Stadium,venue_Barabati_Stadium,venue_Brabourne_Stadium,venue_Buffalo_Park,venue_De_Beers_Diamond_Oval,venue_Dr_DY_Patil_Sports_Academy,venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium,venue_Dubai_International_Cricket_Stadium,venue_Eden_Gardens,venue_Green_Park,venue_Himachal_Pradesh_Cricket_Association_Stadium,venue_Holkar_Cricket_Stadium,venue_JSCA_International_Stadium_Complex,venue_Kingsmead,venue_M_Chinnaswamy_Stadium,venue_MA_Chidambaram_Stadium,venue_Maharashtra_Cricket_Association_Stadium,venue_Narendra_Modi_Stadium,venue_Nehru_Stadium,venue_New_Wanderers_Stadium,venue_Newlands,venue_OUTsurance_Oval,venue_Punjab_Cricket_Association_IS_Bindra_Stadium,venue_Rajiv_Gandhi_International_Stadium,venue_Saurashtra_Cricket_Association_Stadium,venue_Sawai_Mansingh_Stadium,venue_Shaheed_Veer_Narayan_Singh_International_Stadium,venue_Sharjah_Cricket_Stadium,venue_Sheikh_Zayed_Stadium,venue_St_George's_Park,venue_SuperSport_Park,venue_Vidarbha_Cricket_Association_Stadium,venue_Wankhede_Stadium,overs,wickets,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,playoff,knockout,final,final_score,y_test_pred_lr,y_test_pred_la,y_test_pred_ri,y_test_pred_abr,y_test_pred_knr
92758,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,31,48,2,41,2,0,0,0,162,160.568573,169.671660,160.583762,161.652880,161.1
92759,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,32,48,2,37,2,0,0,0,162,159.776337,168.646163,159.791935,159.787940,159.3
92760,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,33,50,2,39,2,0,0,0,162,160.482574,169.476777,160.497553,161.352967,181.0
92761,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,34,50,2,38,2,0,0,0,162,159.486450,168.411264,159.501366,161.352967,185.4
92762,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,35,51,2,39,2,0,0,0,162,159.307526,168.287145,159.322022,163.727391,182.1


In [24]:
test['ensemble_prediction'] = (test['y_test_pred_lr'] + test['y_test_pred_la'] + test['y_test_pred_ri'] + test['y_test_pred_abr'] + 
                               test['y_test_pred_knr']) / 5

In [26]:
test.head()

,year,batting_team_Chennai_Super_Kings,batting_team_Deccan_Chargers,batting_team_Delhi_Capitals,batting_team_Gujarat_Lions,batting_team_Kochi_Tuskers_Kerala,batting_team_Kolkata_Knight_Riders,batting_team_Mumbai_Indians,batting_team_Pune_Warriors,batting_team_Punjab_Kings,batting_team_Rajasthan_Royals,batting_team_Rising_Pune_Supergiant,batting_team_Royal_Challengers_Bangalore,batting_team_Sunrisers_Hyderabad,bowling_team_Chennai_Super_Kings,bowling_team_Deccan_Chargers,bowling_team_Delhi_Capitals,bowling_team_Gujarat_Lions,bowling_team_Kochi_Tuskers_Kerala,bowling_team_Kolkata_Knight_Riders,bowling_team_Mumbai_Indians,bowling_team_Pune_Warriors,bowling_team_Punjab_Kings,bowling_team_Rajasthan_Royals,bowling_team_Rising_Pune_Supergiant,bowling_team_Royal_Challengers_Bangalore,bowling_team_Sunrisers_Hyderabad,toss_decision_bat,toss_decision_field,venue_Arun_Jaitley_Stadium,venue_Barabati_Stadium,venue_Brabourne_Stadium,venue_Buffalo_Park,venue_De_Beers_Diamond_Oval,venue_Dr_DY_Patil_Sports_Academy,venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium,venue_Dubai_International_Cricket_Stadium,venue_Eden_Gardens,venue_Green_Park,venue_Himachal_Pradesh_Cricket_Association_Stadium,venue_Holkar_Cricket_Stadium,venue_JSCA_International_Stadium_Complex,venue_Kingsmead,venue_M_Chinnaswamy_Stadium,venue_MA_Chidambaram_Stadium,venue_Maharashtra_Cricket_Association_Stadium,venue_Narendra_Modi_Stadium,venue_Nehru_Stadium,venue_New_Wanderers_Stadium,venue_Newlands,venue_OUTsurance_Oval,venue_Punjab_Cricket_Association_IS_Bindra_Stadium,venue_Rajiv_Gandhi_International_Stadium,venue_Saurashtra_Cricket_Association_Stadium,venue_Sawai_Mansingh_Stadium,venue_Shaheed_Veer_Narayan_Singh_International_Stadium,venue_Sharjah_Cricket_Stadium,venue_Sheikh_Zayed_Stadium,venue_St_George's_Park,venue_SuperSport_Park,venue_Vidarbha_Cricket_Association_Stadium,venue_Wankhede_Stadium,overs,wickets,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,playoff,knockout,final,final_score,y_test_pred_lr,y_test_pred_la,y_test_pred_ri,y_test_pred_abr,y_test_pred_knr,ensemble_prediction
92758,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,31,48,2,41,2,0,0,0,162,160.568573,169.671660,160.583762,161.652880,161.1,162.715375
92759,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,32,48,2,37,2,0,0,0,162,159.776337,168.646163,159.791935,159.787940,159.3,161.460475
92760,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,33,50,2,39,2,0,0,0,162,160.482574,169.476777,160.497553,161.352967,181.0,166.561974
92761,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,34,50,2,38,2,0,0,0,162,159.486450,168.411264,159.501366,161.352967,185.4,166.830409
92762,2020,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,35,51,2,39,2,0,0,0,162,159.307526,168.287145,159.322022,163.727391,182.1,166.548817


In [27]:
predictions = [y_train_pred_lr, y_train_pred_la, y_train_pred_ri, y_train_pred_abr, y_train_pred_knr]
columns = ['y_train_pred_lr', 'y_train_pred_la', 'y_train_pred_ri', 'y_train_pred_abr', 'y_train_pred_knr']
for col, pred in zip(columns, predictions):
    train[col] = pred

In [25]:
train['ensemble_prediction'] = (train['y_train_pred_lr'] + train['y_train_pred_la'] + train['y_train_pred_ri'] + train['y_train_pred_abr'] + 
                                train['y_train_pred_knr']) / 5

In [29]:
train.head()

,year,batting_team_Chennai_Super_Kings,batting_team_Deccan_Chargers,batting_team_Delhi_Capitals,batting_team_Gujarat_Lions,batting_team_Kochi_Tuskers_Kerala,batting_team_Kolkata_Knight_Riders,batting_team_Mumbai_Indians,batting_team_Pune_Warriors,batting_team_Punjab_Kings,batting_team_Rajasthan_Royals,batting_team_Rising_Pune_Supergiant,batting_team_Royal_Challengers_Bangalore,batting_team_Sunrisers_Hyderabad,bowling_team_Chennai_Super_Kings,bowling_team_Deccan_Chargers,bowling_team_Delhi_Capitals,bowling_team_Gujarat_Lions,bowling_team_Kochi_Tuskers_Kerala,bowling_team_Kolkata_Knight_Riders,bowling_team_Mumbai_Indians,bowling_team_Pune_Warriors,bowling_team_Punjab_Kings,bowling_team_Rajasthan_Royals,bowling_team_Rising_Pune_Supergiant,bowling_team_Royal_Challengers_Bangalore,bowling_team_Sunrisers_Hyderabad,toss_decision_bat,toss_decision_field,venue_Arun_Jaitley_Stadium,venue_Barabati_Stadium,venue_Brabourne_Stadium,venue_Buffalo_Park,venue_De_Beers_Diamond_Oval,venue_Dr_DY_Patil_Sports_Academy,venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium,venue_Dubai_International_Cricket_Stadium,venue_Eden_Gardens,venue_Green_Park,venue_Himachal_Pradesh_Cricket_Association_Stadium,venue_Holkar_Cricket_Stadium,venue_JSCA_International_Stadium_Complex,venue_Kingsmead,venue_M_Chinnaswamy_Stadium,venue_MA_Chidambaram_Stadium,venue_Maharashtra_Cricket_Association_Stadium,venue_Narendra_Modi_Stadium,venue_Nehru_Stadium,venue_New_Wanderers_Stadium,venue_Newlands,venue_OUTsurance_Oval,venue_Punjab_Cricket_Association_IS_Bindra_Stadium,venue_Rajiv_Gandhi_International_Stadium,venue_Saurashtra_Cricket_Association_Stadium,venue_Sawai_Mansingh_Stadium,venue_Shaheed_Veer_Narayan_Singh_International_Stadium,venue_Sharjah_Cricket_Stadium,venue_Sheikh_Zayed_Stadium,venue_St_George's_Park,venue_SuperSport_Park,venue_Vidarbha_Cricket_Association_Stadium,venue_Wankhede_Stadium,overs,wickets,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,playoff,knockout,final,final_score,y_train_pred_lr,y_train_pred_la,y_train_pred_ri,y_train_pred_abr,y_train_pred_knr,ensemble_prediction
32,2008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,31,61,0,59,0,0,0,0,222,195.605927,194.320212,195.617762,178.259753,219.1,196.580731
33,2008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,32,61,1,59,1,0,0,0,222,188.419891,187.451130,188.432337,178.259753,217.0,191.912622
34,2008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,33,61,1,59,1,0,0,0,222,187.355743,186.372279,187.368031,178.259753,224.9,192.851161
35,2008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,34,61,1,59,1,0,0,0,222,186.291656,185.293427,186.303724,178.259753,226.9,192.609712
36,2008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,35,61,1,58,1,0,0,0,222,185.295502,184.227914,185.307537,178.259753,211.2,188.858141


In [26]:
y_ensemble_train = train['ensemble_prediction'].values
y_ensemble_test = test['ensemble_prediction'].values

In [27]:
print('Ensemble Method:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_ensemble_train))
print('Test r2_score: ', r2_score(y_test, y_ensemble_test))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_ensemble_train)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_ensemble_test)))
print('-' * 100)
print('Train MAE: ', mean_absolute_error(y_train, y_ensemble_train))
print('Test MAE: ', mean_absolute_error(y_test, y_ensemble_test))

Ensemble Method:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6911966119312616
Test r2_score:  0.653037163748178
----------------------------------------------------------------------------------------------------
Train RMSE:  16.375343159653003
Test RMSE:  17.842673029000704
----------------------------------------------------------------------------------------------------
Train MAE:  12.05150734480765
Test MAE:  13.790511537257442


In [28]:
((mean_absolute_error(y_train, y_ensemble_train) + mean_absolute_error(y_test, y_ensemble_test)) / 2) / 2

6.460504720516273

In [34]:
for ov in train['overs'].unique():
    tdf_train = train[train['overs'] == ov]
    tdf_test = test[test['overs'] == ov]
    y_ensemble_train_temp = tdf_train['ensemble_prediction'].values
    y_ensemble_test_temp = tdf_test['ensemble_prediction'].values
    y_train_temp = tdf_train['final_score']
    y_test_temp = tdf_test['final_score']
    print('Over: ', ov)
    print('-' * 100)
    print('Train r2_score: ', r2_score(y_train_temp, y_ensemble_train_temp))
    print('Test r2_score: ', r2_score(y_test_temp, y_ensemble_test_temp))
    print('-' * 100)
    print('Train RMSE: ', np.sqrt(mean_squared_error(y_train_temp, y_ensemble_train_temp)))
    print('Test RMSE: ', np.sqrt(mean_squared_error(y_test_temp, y_ensemble_test_temp)))
    print('-' * 100)
    print('Train MAE: ', mean_absolute_error(y_train_temp, y_ensemble_train_temp))
    print('Test MAE: ', mean_absolute_error(y_test_temp, y_ensemble_test_temp))
    print('*' * 100)

Over:  5
----------------------------------------------------------------------------------------------------
Train r2_score:  0.41278337063924586
Test r2_score:  0.3095791226687996
----------------------------------------------------------------------------------------------------
Train RMSE:  23.29948454988079
Test RMSE:  25.130559039214774
----------------------------------------------------------------------------------------------------
Train MAE:  17.843930359999238
Test MAE:  20.71125141803737
****************************************************************************************************
Over:  6
----------------------------------------------------------------------------------------------------
Train r2_score:  0.4620011648931691
Test r2_score:  0.3633153995122952
----------------------------------------------------------------------------------------------------
Train RMSE:  22.25215627327593
Test RMSE:  24.16558337551606
--------------------------------------------------

In [32]:
file = open('linear.pkl', 'wb')
pickle.dump(lr, file)
file.close()

In [29]:
model_names = ['linear_1', 'lasso_1', 'ridge_1', 'ada_boost_1', 'k_neighbors_1']
models = [lr, la, ri, abr, knr]
for model_name, model in zip(model_names, models):
    file = open(model_name + '.pkl', 'wb')
    pickle.dump(model, file)
    file.close()

In [36]:
error = 6.580498382
runs = 179

In [37]:
int(np.round(runs + error, 0))

186